# A toy example for modeling complex drug-protein interactions using RAF kinases and RAF inhibitors

Here, we provide the step-by-step construction code for a toy example to model complex drug-protein interactions using PySB with energy formulation (through support for eBNG). This example describes RAF kinases as the drug target and RAF inhibitors as the drug. To run this code you'll need to have Pysb with BNG installed, please follow instructions at: http://pysb.org/ .

### Manual definition of the biochemical reaction system

To start, we import all required Pysb classes and instantiate the model:

In [1]:
from pysb import Model, Monomer, Parameter, Expression,  Rule, Observable, Initial, Annotation, EnergyPattern, ANY
from pysb.bng import generate_equations
from pysb.export import export
from sympy import exp, log

Model();

Next, we define the two basic components of the model, RAF kinases (R) and RAF inhibitors (I): 

In [2]:
#define a monomer R that represents a RAF kinase with a binding site for RAF (r) and another for the drug (i)
Monomer('R', ['r', 'i']);   
#define a monomer I that represents a RAF inhibitor with a binding site for RAF (r) 
Monomer('I',['r']);   

We define the parameters for initializing abundance of components:

In [3]:
#define the initial conditions for R and I
Parameter('R_0',0.01);  # uM
Parameter('I_0',0.0);  # uM
Initial(R(r=None, i=None), R_0);
Initial(I(r=None), I_0);

Then, we define the kinetic parameters and thermodynamic factors: 

In [4]:
#define dissociation constant (kD), forward rate (kf) and distributionr rate (phi) for RAF dimerization
Parameter('RR_kr',10);  #/s
Parameter('RR_kf',1.0);  #/s/uM
Parameter('RR_phi',1.0); #unitless 

#define dissociation constant (kD), forward rate (kf) and distributionr rate (phi) for drug binding to RAF
Parameter('RI_kr',0.1);  #/s
Parameter('RI_kf',1.0);  #/s/uM
Parameter('RI_phi',1.0); #unitless

#define thermodynamic factors f and g
Parameter('f',1.0);  #unitless 
Parameter('g',1.0);  #unitless

We convert the kinetic parameters into corresponding energy parameters:

In [5]:
#convert kinetic parameters into energies for RAF dimerization
Expression('Gf_RR', log(RR_kr/RR_kf)); #unitless 
Expression('Ea0_RR',-RR_phi*log(RR_kr/RR_kf)-log(RR_kf)); #unitless 

#convert kinetic parameters into energies for drug binding to RAF
Expression('Gf_RI', log(RI_kr/RI_kf)); #unitless 
Expression('Ea0_RI',-RI_phi*log(RI_kr/RI_kf)-log(RI_kf)); #unitless 

#convert thermodynamic factors into energies
Expression('Gf_f',log(f)); #unitless 
Expression('Gf_g',log(g)); #unitless 

We define the energy patterns to assign energies within biochemical species:

In [6]:
# define energy in bond between R and R
EnergyPattern('ep_RR',R(r=1)%R(r=1),Gf_RR);

# define energy in bond between R and I
EnergyPattern('ep_RI',R(i=1)%I(r=1),Gf_RI);

# define additional energy in bond betwee RAF dimer and a single drug molecule
Expression('Gf_RRI',Gf_f);
EnergyPattern('ep_RRI',R(r=1,i=None)%R(r=1,i=2)%I(r=2), Gf_RRI);

# define additional energy in bond betwee RAF dimer and two drug molecules
Expression('Gf_IRRI', Gf_f + Gf_g);
EnergyPattern('ep_IRRI',I(r=2)%R(r=1,i=2)%R(r=1,i=3)%I(r=3), Gf_IRRI);

We define observables that are used later to visualize results from model simulations: 

In [7]:
# define observable for total RAF and total drug
Observable('Rtot_obs', R());
Observable('Itot_obs', I());

#define an observable that counts the amount of active RAF when RAF represents a BRAF V600E/K mutant
#that is active independently of dimerization stauts (i.e. both as a monomer or as a dimer) as long as it is not drug bound
Observable('R_BRAFmut_active_obs', R(i=None));

#define an observable that counts the amount of active RAF when RAF here represents a wild type version of BRAF or CRAF
#that is active only when dimerized and not drug bound
Observable('R_RAFwt_active_obs', R(r=1,i=None)%R(r=1));

# define observable for drug unbound RAF monomer
Observable('R_obs', R(i=None,r=None));

# define observable for RAF dimer unbound by drug
Observable('RR_obs', R(r=1,i=None)%R(r=1,i=None));

# define observable for RAF dimer bound by single drug
Observable('RRI_obs', R(r=1,i=None)%R(r=1,i=2)%I(r=2));

# define observable for RAF dimer bound by double drug
Observable('IRRI_obs', I(r=2)%R(r=1,i=2)%R(r=1,i=3)%I(r=3));

As the last step in the model construction, we define the reactions for RAF dimerization and drug binding: 

In [8]:
#define RAF dimerization reaction
Rule('RR', R(r=None)+R(r=None) | R(r=1)%R(r=1) , RR_phi, Ea0_RR, energy=True);

#define drug binding to RAF reaction
Rule('RI', R(i=None)+I(r=None) | R(i=1)%I(r=1) , RI_phi, Ea0_RI, energy=True);

### Automatic generation of the kinetic model

We generate the kinetic model by passing the information build via PySB to BNG, parse the returned reaction network and list the properties of the resulting kinetic model: 

In [9]:
# generate the model equations
generate_equations(model)

# print model infomration
print ('RAF-RAFi model information')
print ('Species:',len(model.species))
print ('Parameters:',len(model.parameters)+len(model.initial_conditions))
print ('Expressions:',len(model.expressions))
print ('Observables:', len(model.observables))
ntotr=len(model.rules);
nenergy=len([r for r in model.rules if r.energy]);
print ('Total Rules:', ntotr)
print ('Energy Rules:', nenergy)
print('Non-energy Rules:', ntotr-nenergy)
print('Energy Patterns:', len(model.energypatterns))
print('Reactions:',len(model.reactions))

RAF-RAFi model information
Species: 6
Parameters: 12
Expressions: 8
Observables: 8
Total Rules: 2
Energy Rules: 2
Non-energy Rules: 0
Energy Patterns: 4
Reactions: 12


We plot the automatically generated equation rates to check that thermodynamic factors indeed control cooperative reaction rates:

In [10]:
# to appear

### Model simulation of drug-dose response at steady state

We use the generated model to simulate the response of RAF kinases to three classes of RAF inhibitors: 1st generation (e.g. Vemurafenib, Dabrafenib and Encorafenib), paradox breakers (e.g. PLX8349) and panRAF (e.g. LY3009120, AZ628) inhibitors. We compare the results to a hypothetical RAF inhibitor that has no cooperative effect with RAF dimerization (f and g termodynamic parameters are unity so they do not impose any extra energy). We observe the effect of the drugs in situations with low and high propensity for RAF dimerization (controlled by setting the Kd values of the RAF dimerization reactions) to study the effect that the drugs have in absence or presence of the dimerization status of RAFs, for example as induced by Ras-GTP signal that induces RAF dimerization. This analysis is run to steady state, meaning that the drug-dose response represent the inhibitory response achieved when all reactions are let to run until they equilibrate.First, we set up the model with the right parameter values and with a drug-dose response concentrations of the RAF inhibitor: 

In [11]:
#import ODE simulator
from pysb.simulator import ScipyOdeSimulator
#import various utility packages
import numpy as np
import pandas as pd

#set the dilution range for the RAF inhibitor
RAFi_dil=np.logspace(-4, 1, 20, base=10.0); #uM
#set the values of f and g to model RAF inhibitors with different complex drug-protein interactions
#independent: f=1, g=1 , 1st generation: f= 0.001, g=1000; paradox breaker: f= 1.0, g=1000; panRAF: f= 0.001, g=1
ff=[1.0, 0.001, 1.0, 0.001];
gg=[1.0, 1000, 1000, 1];
fgtitle=['non-cooperative', '1st_gen', 'paradox_breaker', 'panRAF'];
#set the Kd values to use for baseline RAF dimerization to simulate Ras-GTP signaling that induced dimerization
RR_kfs_exp=np.linspace(-1, 5, 13);
RR_kfs=10**RR_kfs_exp;

#set up the ODE simulator for the model
sim = ScipyOdeSimulator(model);

Then, we define functions to run to the simulations to steady state:

In [12]:
def equilibrate(simulator, initials=None, param_values=None):
    """Simulate a model from given initial conditions until it reaches steady state"""
    scale = 10
    t_start = 100
    df = None
    tspan = np.geomspace(t_start, t_start * scale)
    while True:
        res = simulator.run(tspan=tspan, initials=initials, param_values=param_values)
        initials = res.species[-1]
        close = np.isclose(
            *res.species[[-1,-2]].view(float).reshape(2,-1),
            rtol=1e-3
        )
        cs = np.sum(close)
        n = len(simulator.model.species)
        if np.all(close):
            break
        tspan *= scale
    return res

Then, we perform multiple simulations of the systems at each defined combination of thermodynamic parameters (f,g), RAF inhibitor concentration (RAFi) and RAF dimerization baseline (RR_Kd): 

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import math
from tqdm.notebook import tqdm, trange

#create a bar to keep track of simulation progress
p_bar_sim = tqdm(desc='Simulation progress', total=len(ff)*len(RR_kfs)*len(RAFi_dil))

#define observables to plot
plt_obs=['R_BRAFmut_active_obs', 'R_RAFwt_active_obs', 'R_obs', 'RR_obs', 'RRI_obs', 'IRRI_obs'];
plot_obs_names=['RAF_mut', 'RAF_wt', 'R', 'RR', 'RRI', 'IRRI'];

#define figure 
fig, ax = plt.subplots(len(plt_obs),len(gg), sharey=True);
fig.suptitle("Simulations of RAF inhibitor dose-response effect on RAF activity");  

#define plot colors
cmap=plt.get_cmap('copper');
col=cmap(np.linspace(0.0, 1.0, len(RR_kfs_exp)))[::-1];

#simulate the different parameter combinations
ss_v = np.empty([len(RAFi_dil), len(plt_obs)]);
for i in range (len(ff)):
    for j in range(len(RR_kfs)):
        for k in range(len(RAFi_dil)):
            #run simulation with modified parameters
            param_values={'f': ff[i] ,'g': gg[i], 'RR_kf': RR_kfs[j], 'I_0': RAFi_dil[k]};
            #run this to assure model is run to steady state, 
            res=equilibrate(sim, param_values=param_values)
            #update progress
            p_bar_sim.update(1);
            #extract end of simulation for each osbervables from the dataframe of simulation results
            for z in range(len(plt_obs)):
                ss_v[k,z]=res.observables[plt_obs[z]][-1];
                
        #plot the results for a given RR_KD and f,g combination
        for z in range(len(plt_obs)):
             #plot simualtion 
             h=ax[z,i].plot(RAFi_dil, ss_v[:,z], color = col[j,:]);   
             ax[z,i].set_xscale('log');
             #set axis names
             if (i==0):
                ax[z,i].set_ylabel(plot_obs_names[z]);
             if (z==0):
                ax[z,i].title.set_text(fgtitle[i]);
             if (z==(len(plt_obs)-1)):
                ax[z,i].set_xlabel('RAFi (uM)');
             else:
                ax[z,i].set_xticklabels([]);
                
#add legend
fig.legend(ax, labels= list(map(str, math.log(model.parameters['RR_kr'].value,10)-RR_kfs_exp)) , bbox_to_anchor=(1.04,1), loc="upper right", borderaxespad=0.1,
           title="RR_kDs (log10)");

<IPython.core.display.Javascript object>

The resulting simulations show the expected behavior of these four different RAF inhibitor classes on both the situation of RAF representing a BRAF V600E/K mutant or wild-type BRAF/CRAF:

* **Non-cooperative RAF inhibitor** - In the first column, the independent RAF inhibitor does not change its affinity to RAF depending on dimerization status, nor does it influence dimerization itself. As such, BRAF mutant inhibition is effective (first row) and independent on RAF dimerization from upstream signal (RR_kD). Similarly, the effect on a wild-type RAF context is a straightforward dose-dependent inhibition of RAF signaling that depends on the baseline RAF dimerization due to upstream signal (RR_kD) (second row). Note that there is no difference in the potency of the drug between the oncogenic BRAF mutant and RAF wild-type signal (compare first and second panel). 


* **1st generation RAF inhibitor** - In the second column, the 1st generation RAF inhibitor efficiently inhibits RAF signaling from monomers (light colored line), but RAF dimeric signal generated by either upstream RAF dimerization (RR_Kd) or by the propensity of the drug to cause RAF dimerization (f parameter less than one) creates a resistance mechanism (seen as the increased amount of active RAFs in the first row when comparing light and dark colored lines). This resistance mechanism is due to the low affinity of the drug for the second RAF protomer in a RAF dimer (g parameter more than one), as seen by the rise of single drug-bound RAF dimers in the 5th row. In addition, wild-type RAF signaling is potentiated by the drug, as can be seen by the increase in active RAF signaling (2nd row) that is drug-dependent at otherwise low levels of upstream RAF dimerization. This effect, known to induced toxicity through inducing proliferation of other malignant cells by drug-induced MAPK activation, is also known as paradoxical activation. 


* **Paradox-breaker RAF inhibitor** - In the third column, a paradox breaker RAF inhibitor is seen reducing the extend of resistance due to RAF dimerization (1st row) in the BRAF-mutant case and of RAF dimerization potentiation by paradoxical activation (2nd row), since the drug binding does not induce dimerization (f is equal to one) and thus does not synergize with the upstream RAF dimerization signal. This can be seen as the reduced amount of single drug-bound RAF dimers in the 5th row. 


* **panRAF RAF inhibitor** - In the last column, a panRAF inhibitor is seen eventually binding both protomers in RAF dimers (g is equal to one), thus completely ablating any resistance mechanism caused by RAF dimerization. This can be seen as the eventual reduction in the amount of single drug-bound RAF dimers in the 5th row and dose-dependent rise of double drug-bound RAF dimers in the 6th column. Note that in this case, pan RAF inhibitors have the same potency on the RAF-mutant and wild-type signal again (compare potencies on 1st and 2nd rows).


Thus, the model automatically generated using energy-based rule-based modelling properly describes and simulates the complex drug-protein interactions that are supposed to govern drug efficacy and toxicity in this simplified scenario for RAF kinases and RAF inhibitors.



### Model simulation of temporal dynamics during drug treatment

The previous simulations analyzed the drug-dose response of RAF to RAF inhibitors applied for long-term drug treatment at fixed conditions (e.g. fixed level of RAF dimerization baseline). However, drugs operate on proteins that often experience temporal  perturbations, as for example from the activation of an upstream signal. In this second example, we simulate the behaviour of RAF inhibitors having different energetic properties during temporal dynamic perturbations in RAF dimerization. 

First, we generate the dynamic profile for the addtion of a RAF inhibitor and for a subsequent series of square pulses of increased RAF dimerization (through the k forward rate Kf_RR): 

In [ ]:
from pysb.core import as_complex_pattern

#define the train of square pulses that define temporal dynamic control over RAF dimerization (Kf_RR)
ncycle=6;
ton=40;
toff=50;
toset=10;
Kf_RR_min=10**-2;
Kf_RR_max=10**2.5;

#set the concentration and time of addition of the RAFi inhibitor
RAFi_conc_init=0.0;
RAFi_conc_add=10.0;
RAFi_time_add=20;


#generate list of events that dynamically change RAFi 
t_events=[0.0, RAFi_time_add];
events=['I_0', 'I_0'];
events_value=[RAFi_conc_init, RAFi_conc_add];

#generate list of events that dynamically change Kf_RR
for i in range(ncycle):
    t_events= t_events + [t_events[-1] + toff + toset] + [t_events[-1] + ton + toff + toset] ;
    events= events + ['RR_kf'] + ['RR_kf'];
    events_value= events_value + [Kf_RR_max] + [Kf_RR_min];
    toset=0;
    
#generate dynamic signals for RAFi and Kf_RR
t_Kf_RR_dyn=[0.0];
Kf_RR_dyn=[Kf_RR_min];
t_RAFi=[0.0];
RAFi_dyn=[RAFi_conc_init];
for i in range(len(events)):
    if (events[i]=='RR_kf'):
        t_Kf_RR_dyn= t_Kf_RR_dyn + [t_events[i]] + [t_events[i]];
        Kf_RR_dyn= Kf_RR_dyn + [Kf_RR_dyn[-1]] + [events_value[i]];
    elif (events[i]=='I_0'):
        t_RAFi= t_RAFi + [t_events[i]] + [t_events[i]];
        RAFi_dyn= RAFi_dyn + [RAFi_dyn[-1]] + [events_value[i]];
t_RAFi= t_RAFi + [t_Kf_RR_dyn[-1]];
RAFi_dyn = RAFi_dyn + [RAFi_dyn[-1]];


Next, we define the energetic properties of two 1st generation RAF inhibitors which have the same kinetic rates but in which the cooperativity with RAF dimerization is assigned either to the forward (RR_phi=1.0) or backward rate (RR_phi=0.0). This will change how long lived are RAF dimers once induced by the dynamic pulses of increase RAF dimerization. 

In [ ]:
#define energy parameters for the RAF inhibitors to be simulated
ff=[0.001, 0.001];
gg=[1000, 1000];
RI_phi=[1.0, 1.0];
RR_phi=[1.0, 0.0];
lgn_dyn=['RAFi','Kd_RR', 'RR_phi=1', 'RR_phi=0'];
        
#create figure for plotting 
fig, ax = plt.subplots(2+len(plt_obs),1);
#plot dynamic RAFi concentration
ax[0].plot(t_RAFi, RAFi_dyn, color='k');
ax[0].set_ylabel('RAFi (I)');
ax[0].set_xticklabels([]);
#plot dynamic Kd_RR rate
ax[1].plot(t_Kf_RR_dyn, Kf_RR_dyn, color='r');
ax[1].set_ylabel('Kf_RR');
ax[1].set_xticklabels([]);

#for each RAF inhibitor to be simulated
for i in range (len(ff)):
    #set up kinetic parameters and initial conditions
    param_values={'f': ff[i] ,'g': gg[i], 'RI_phi':RI_phi[i], 'RR_phi':RR_phi[i], 'RR_kf': Kf_RR_dyn[0], 'I_0': RAFi_dyn[0]};
    #run it to steady state before running drug addition and Kd_RR pulse train
    res=equilibrate(sim, param_values=param_values);
    #run consecutive simulations updating conditions according to events 
    #(drug addition, changes in Kd_RR) pulse train to set KdRR values
    res_obs=res.observables[plt_obs][-1];
    t=[0.0];
    
    for j in range(len(events)-1):
        #save the state of the previous simulation to restart at same point
        initials_pre = res.dataframe.iloc[-1, :len(model.species)].copy();
        #create the tspan for simulation from these event to the next
        tspan= np.linspace(t_events[j], t_events[j+1]);
        #update param values with the event value
        param_values[events[j]]=events_value[j];
        #if the drug changed, set it in the current species states
        if (events[j]=="I_0"):
            #get index of inhibitor specie
            i_I_0=model.get_species_index(as_complex_pattern(model.monomers.I(r=None)));
            initials_pre[i_I_0]=events_value[j];        
        #run the simulation for the necessary time (until next event)
        res=sim.run(tspan=tspan, param_values=param_values, initials=np.array(initials_pre));
        #append the observables
        res_obs=np.append(res_obs[plt_obs],res.observables[plt_obs]); 
        #append simulation time
        t=np.append(t,tspan);
        
    #plot the results for a given setting of parameters
    for z in range(len(plt_obs)):
             #set same yaxes
             if (z>0):
                ax[z].get_shared_y_axes().join(ax[2], ax[z+2]);
             #plot simualtion 
             h=ax[z+2].plot(t, res_obs[plt_obs[z]]);   
             #set axis names
             ax[z+2].set_ylabel(plot_obs_names[z]);
             if (z==(len(plt_obs)-1)):
                ax[z+2].set_xlabel('Time (s)');
             else:
                ax[z+2].set_xticklabels([]);
#add legend
fig.legend(ax, labels=lgn_dyn , loc="upper right", borderaxespad=0.1);           

The simulation shows that distribution rates, which define how changes in cooperativity affect forward or backward rates, can greatly influce drug efficacy during dynamic perturations. In this case, two 1st generation RAF inhibitors with different distribution rates, but having the exact same forward and backward rates, behave very differently when a dynamic perturbation in RAF dimerization is applied the system. By changing the distribution rate from RR_phi=0 to RR_phi=1, the cooperativity imposed between drug binding and RAF dimerization shifts from controlling the rate of RAF dissasembly to control the rate of RAF assembly. In the case of RR_phi=0, the drug binding to RAF dimers slows their disassembly after the dimerization signal disappears, whose creating a continuous RAF signal that can cause resistance (RAF_mut, second row) or toxicity (RAF_wt). Instead, in the case of RR_ph=1, the drug binding to RAF dimers increases the speed of their assembly when the dimerization signal appears. This causes a slight faster induction of RAF dimers, but removes the continous activation seen in the previous case. The beahviours drugs with complex interactions with their targeted proteins is thus influenced not just by their basic forward and backward rates but also by the way in which cooperativity modifies the complex rates of multiprotein-drug complexes.